In [1]:
# General libraries
import pandas as pd
import numpy as np

# Scikit Learn libraries
from sklearn import ensemble
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Scipy libraries
from scipy import stats
import joblib
import warnings
warnings.filterwarnings('ignore')

# Load Dataset

In [2]:
folder_path = "../data/"

data_path = folder_path + "complex_processed_data.csv"
standardized_data_path = folder_path + 'complex_processed_standardized_data.csv'
standardized_poutliers_removed_data_path = folder_path + 'complex_processed_standardized_outliers_removed_data.csv'

df_solubility = pd.read_csv(standardized_data_path)

# Process Dataset

Process Dataset before the model creation.
The following actions were done:
* Split the independent variable from the dependent ones;
* Split Dataset for training and testing.

In [3]:
# Split dataset into X and Y for machine learning

df_sol_X = df_solubility.copy()
df_sol_X.drop(columns=['solubility'], axis=1, inplace=True)

df_sol_y = df_solubility[['solubility']]

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
                        df_sol_X, df_sol_y, 
                        train_size = 0.8,
                        test_size = 0.2,
                        random_state = 10
                        )

# Gradient Boosting regression - XGBoost

In [5]:
# Define functions
def get_adj_r2(n_observations, n_independent_variables, r2_score):
    Adj_r2 = 1 - ((1 - r2_score) * (n_observations - 1)) / (n_observations - n_independent_variables - 1)
    return Adj_r2

In [6]:
# The cross validation scheme to be used for train and test
cv = 10
folds = KFold(n_splits = cv, shuffle = True, random_state = 100)

In [7]:
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

xgboost = ensemble.GradientBoostingRegressor(**params)
xgboost.fit(x_train, y_train)

r2 = r2_score(y_test, xgboost.predict(x_test))
print("The r2 score on test set: {:.4f}".format(r2))

The r2 score on test set: 0.1369


In [8]:
# Specify range of hyperparameters to tune
hyper_params = {
    'n_estimators':[100, 200, 300, 400, 500],
    'max_depth':[2, 3, 4,5],
    "min_samples_split": [1,2,3,4],
    "min_samples_leaf": [1,1.5,2],
    "learning_rate": [0.01,0.02,0.03,0.4,0.5],
    "loss": ["squared_error", "absolute_error", "huber", "quantile"],
    #"criterion": ["friedman_mse", "squared_error", "mse"],
    }


# Call GridSearchCV()
model_cv = GridSearchCV(
    estimator = ensemble.GradientBoostingRegressor(),
    param_grid = hyper_params,
    scoring= 'r2',
    cv = folds,
    verbose = 1,
    return_train_score=True,
    n_jobs = -1,
    refit = True
    )


# Fit the model
best_model = model_cv.fit(x_train, np.ravel(y_train)) 

print(model_cv.best_params_)

Fitting 10 folds for each of 4800 candidates, totalling 48000 fits
{'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 300}


In [9]:
# Create new model with best_params_ from grid search
# Use cross validation on the best_params_ model

xgboost_best = ensemble.GradientBoostingRegressor(
    n_estimators=model_cv.best_params_['n_estimators'],
    max_depth=model_cv.best_params_['max_depth'],
    min_samples_split=model_cv.best_params_['min_samples_split'],
    min_samples_leaf=model_cv.best_params_['min_samples_leaf'],
    learning_rate=model_cv.best_params_['learning_rate'],
    loss=model_cv.best_params_['loss']
    )

xgboost_best.fit(x_train, y_train)

GradientBoostingRegressor(learning_rate=0.01, max_depth=2, min_samples_split=3,
                          n_estimators=300)

In [10]:
r2 = r2_score(y_test, xgboost_best.predict(x_test))
print("The r2 score on test set: {:.4f}".format(r2))

The r2 score on test set: 0.2572


# Saving trained model

In [11]:
filename = '../models/xgboost_model.joblib'
joblib.dump(xgboost_best, filename)

['../models/xgboost_model.joblib']

In [12]:
def five_two(reg1, reg2, X, y, metric='default'):

  # Choose seeds for each 2-fold iterations
  seeds = [13, 51, 137, 24659, 347]

  # Initialize the score difference for the 1st fold of the 1st iteration 
  p_1_1 = 0.0

  # Initialize a place holder for the variance estimate
  s_sqr = 0.0

  # Initialize scores list for both classifiers
  scores_1 = []
  scores_2 = []
  diff_scores = []

  # Iterate through 5 2-fold CV
  for i_s, seed in enumerate(seeds):

    # Split the dataset in 2 parts with the current seed
    folds = KFold(n_splits=2, shuffle=True, random_state=seed)

    # Initialize score differences
    p_i = np.zeros(2)

    # Go through the current 2 fold
    for i_f, (trn_idx, val_idx) in enumerate(folds.split(X)):
      # Split the data
      trn_x, trn_y = X.iloc[trn_idx], y.iloc[trn_idx]
      val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]

      # Train regression
      reg1.fit(trn_x, trn_y)
      reg2.fit(trn_x, trn_y)

      # Compute scores
      preds_1 = reg1.predict(val_x)
      score_1 = r2_score(val_y, preds_1)
      
      preds_2 = reg2.predict(val_x)
      score_2 = r2_score(val_y, preds_2)

      if metric == "adj_r2":
        score_1 = base_train_adj_r2 = get_adj_r2(
          n_observations=len(trn_y) / 2,
          n_independent_variables=trn_x.shape[1],
          r2_score = score_1
        )

        score_2 = base_train_adj_r2 = get_adj_r2(
          n_observations=len(trn_y) / 2,
          n_independent_variables=trn_x.shape[1],
          r2_score = score_2
        )


      # keep score history for mean and stdev calculation
      scores_1.append(score_1)
      scores_2.append(score_2)
      diff_scores.append(score_1 - score_2)
      print("Fold %2d score difference = %.6f" % (i_f + 1, score_1 - score_2))

      # Compute score difference for current fold  
      p_i[i_f] = score_1 - score_2

      # Keep the score difference of the 1st iteration and 1st fold
      if (i_s == 0) & (i_f == 0):
        p_1_1 = p_i[i_f]

    # Compute mean of scores difference for the current 2-fold CV
    p_i_bar = (p_i[0] + p_i[1]) / 2

    # Compute the variance estimate for the current 2-fold CV
    s_i_sqr = (p_i[0] - p_i_bar) ** 2 + (p_i[1] - p_i_bar) ** 2 

    # Add up to the overall variance
    s_sqr += s_i_sqr
    
  # Compute t value as the first difference divided by the square root of variance estimate
  t_bar = p_1_1 / ((s_sqr / 5) ** .5) 

  print("Regression 1 mean score and stdev : %.6f + %.6f" % (np.mean(scores_1), np.std(scores_1)))
  print("Regression 2 mean score and stdev : %.6f + %.6f" % (np.mean(scores_2), np.std(scores_2)))
  print("Score difference mean + stdev : %.6f + %.6f" 
        % (np.mean(diff_scores), np.std(diff_scores)))
  print("t_value for the current test is %.6f" % t_bar)

In [17]:
five_two(
    reg1=xgboost,
    reg2=xgboost_best,
    X=df_sol_X,
    y=df_sol_y
)

Fold  1 score difference = -0.006949
Fold  2 score difference = -0.063067
Fold  1 score difference = -0.141070
Fold  2 score difference = -0.016634
Fold  1 score difference = -0.125163
Fold  2 score difference = -0.059041
Fold  1 score difference = -0.031952
Fold  2 score difference = -0.058188
Fold  1 score difference = -0.038647
Fold  2 score difference = -0.130422
Regression 1 mean score and stdev : 0.131042 + 0.051056
Regression 2 mean score and stdev : 0.198155 + 0.038910
Score difference mean + stdev : -0.067113 + 0.046069
t_value for the current test is -0.122626


In [29]:
from sklearn.model_selection import cross_val_score
cross_val_score(estimator=xgboost_best, X=df_sol_X, y=df_sol_y, cv=folds, scoring='r2')

array([ 0.20444563,  0.26215949,  0.23695625,  0.3413003 ,  0.24612715,
        0.26305958, -0.14225403,  0.15206445,  0.26732598,  0.20390257])

In [30]:
cross_val_score(estimator=xgboost, X=df_sol_X, y=df_sol_y, cv=folds, scoring='r2')

array([ 0.10325867,  0.15040459,  0.17747865,  0.12050908,  0.20986007,
        0.22875968, -0.30601322,  0.09060705,  0.13289577,  0.15118151])